### Урок 4. Тематическое моделирование. EM-алгоритм

#### Домашнее задание

1. объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов
2. провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать
3. сделать визуализацию кластеров тематик
4. проинтерпритировать получившиеся тематики

In [1]:
# !pip install corus

In [2]:
# !pip install pyLDAvis

In [3]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from tqdm.notebook import tqdm
from tqdm import notebook

import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from nltk.corpus import stopwords
from multiprocessing import Pool
from pymystem3 import Mystem
import pymorphy2

from gensim.models import *
from gensim import corpora

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\AVB\anaconda3\lib\site-packages\matplotlib\__init__.py:169: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
C:\Users\AVB\anaconda3\lib\site-packages\setuptools\_distutils\version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
C:\Users\AVB\anaconda3\lib\site-packages\matplotlib\__init__.py:169: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
C:\Users\AVB\anaconda3\lib\site-packages\setuptools\_distutils\version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
C:\Users\AVB\anaconda3\lib\site-packages\matplotlib\__init__.py:169: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVers

#### Объеденяем 2 датасета в 1 выборку и делаем предобработку текста как для дз к 3 уроку

In [4]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
data = pd.concat([positive, negative])
data

,text
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3,"RT @digger2912: ""Кто то в углу сидит и погибае..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...
...,...
111918,Но не каждый хочет что то исправлять:( http://...
111919,скучаю так :-( только @taaannyaaa вправляет мо...
111920,"Вот и в школу, в говно это идти уже надо("
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :..."


#### Предобработка текста

In [5]:
def user_delete_re_sub (pattern, repl, string):
    return re.sub(pattern, repl, string)

data.text = np.vectorize(user_delete_re_sub)('@[\w]*', '', data.text)
data.text = np.vectorize(user_delete_re_sub)('http*', '', data.text)
data.text = np.vectorize(user_delete_re_sub)('это*', '', data.text)
data.text = np.vectorize(user_delete_re_sub)('всё*', '', data.text)
data.text = np.vectorize(user_delete_re_sub)('ещё*', '', data.text)
data.text = np.vectorize(user_delete_re_sub)('знать*', '', data.text)
data.text = np.vectorize(user_delete_re_sub)('посмотреть*', '', data.text)
data.text = np.vectorize(user_delete_re_sub)('сой*', '', data.text)

data.head(5)

<>:4: DeprecationWarning: invalid escape sequence \w
<>:4: DeprecationWarning: invalid escape sequence \w
C:\Users\AVB\AppData\Local\Temp/ipykernel_17624/751865316.py:4: DeprecationWarning: invalid escape sequence \w
  data.text = np.vectorize(user_delete_re_sub)('@[\w]*', '', data.text)


,text
0,"хоть я и школота, но поверь, у нас то же само..."
1,"Да, е-таки он немного похож на него. Но мой ма..."
2,RT : Ну ты идиотка) я испугалась за тебя!!!
3,"RT : ""Кто то в углу сидит и погибает от голода..."
4,"Вот что значит страшилка :D\nНо блин,посмотре..."


In [6]:
morph = pymorphy2.MorphAnalyzer()
words_regex = re.compile('\w+')

def find_words(text, regex = words_regex):
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]


stopwords_list = stopwords.words('russian')

def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords 
            and w.isalpha()]

def preprocess(text):
    return (lemmatize(find_words(text)))

<>:2: DeprecationWarning: invalid escape sequence \w
<>:2: DeprecationWarning: invalid escape sequence \w
C:\Users\AVB\AppData\Local\Temp/ipykernel_17624/4135999088.py:2: DeprecationWarning: invalid escape sequence \w
  words_regex = re.compile('\w+')
C:\Users\AVB\anaconda3\lib\site-packages\pymorphy2\units\base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
C:\Users\AVB\anaconda3\lib\site-packages\pymorphy2\units\base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
C:\Users\AVB\anaconda3\lib\site-packages\pymorphy2\units\base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect

In [7]:
data.text.iloc[777]

' нет...предупреждает, что их власть надолго...но думаю опять переоценил он свои возможности..алкоголь не даст дожить до 70))'

In [8]:
print(preprocess(data.text.iloc[777]))

['предупреждать', 'власть', 'надолго', 'думать', 'переоценить', 'свой', 'возможность', 'алкоголь', 'дать', 'дожить']


In [9]:
# оставляем случайные 10000 текстов, чтобы ускорить вычисления
data = data.sample(10000)

In [10]:
preprocessed_text = list(tqdm(map(preprocess, data['text']), total=len(data)))

  0%|          | 0/10000 [00:00<?, ?it/s]

In [11]:
data['text'] = preprocessed_text
data.sample(3)

,text
62880,"[набить, тат, мама, увидеть, ибо, жить, хотеть]"
17358,"[старый, новый, партия, lacrachlct]"
51581,"[хотеть, отдых, устать, нереально]"


#### Модель LDA

In [12]:
dictionary = corpora.Dictionary(data['text'])

dictionary.filter_extremes(no_below = 3, no_above = 0.9, keep_n=None) # игнорируем слова, которые встречаются реже 3 раз или составляют более 0.9 словаря 
dictionary.save('tweeter.dict')

In [13]:
corpus = [dictionary.doc2bow(text) for text in data['text']]
corpora.MmCorpus.serialize('tweeter.model', corpus) 

In [14]:
lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=20, chunksize=50, update_every=1, passes=2)

#### Посмотрим 5 основных тематик о которых говорят в твитах

In [15]:
lda.show_topics(num_topics=5, num_words=5, formatted=True)

[(15,
  '0.464*"день" + 0.125*"идти" + 0.085*"песня" + 0.067*"рождение" + 0.037*"домой"'),
 (2,
  '0.357*"почему" + 0.170*"время" + 0.073*"глаз" + 0.060*"понимать" + 0.056*"ехать"'),
 (3,
  '0.000*"маршрутка" + 0.000*"готовить" + 0.000*"сдохнуть" + 0.000*"инф" + 0.000*"плечо"'),
 (4,
  '0.384*"думать" + 0.080*"учить" + 0.077*"русский" + 0.073*"решить" + 0.019*"язык"'),
 (16,
  '0.236*"мочь" + 0.161*"сделать" + 0.091*"это" + 0.074*"помнить" + 0.055*"второй"')]

#### Визуализируем кластеры тематик

In [16]:
vis_data = gensimvis.prepare(lda, corpus, dictionary, mds='mmds')
pyLDAvis.display(vis_data)

C:\Users\AVB\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


#### Интерпретируем получившиеся тематики

#### Тема 1
Ключевае слова: мочь, сделать, неделя, ситуация

Тема - работа

#### Тема 2
Ключевае слова: сегодня, настроение, вечер, любовь

Тема - досуг, развлечения

#### Тема 3
Ключевае слова: свой, любить, нравиться, плакать

Тема - отношения, любовь

И т.д.